In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [3]:

from sklearn.preprocessing import LabelEncoder

def encode_categorical(df, cat_cols):
    df_encoded = df.copy()
    encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        encoders[col] = le
    return df_encoded, encoders


In [4]:
# ===== Load data =====
red = pd.read_csv("Red.csv")
white = pd.read_csv("White.csv")
rose = pd.read_csv("Rose.csv")
sparkling = pd.read_csv("Sparkling.csv")
varieties = pd.read_csv("Varieties.csv")

# Assign WineStyle and combine
red["WineStyle"]       = "Red"
white["WineStyle"]     = "White"
rose["WineStyle"]      = "Rose"
sparkling["WineStyle"] = "Sparkling"
df = pd.concat([red, white, rose, sparkling], ignore_index=True)

# Impute Age
df["IsNV"] = df["Year"] == "N.V."
valid_year = df[df["Year"] != "N.V."].copy()
valid_year["Year"] = valid_year["Year"].astype(int)
valid_year["Age"]  = 2025 - valid_year["Year"]
avg_age = valid_year.groupby("WineStyle")["Age"].mean()

def impute_age(row):
    if row["IsNV"]:
        return avg_age.get(row["WineStyle"], valid_year["Age"].mean())
    try:
        return 2025 - int(row["Year"])
    except:
        return np.nan

df["Age"] = df.apply(impute_age, axis=1)

# Extract Variety
variety_list = varieties["Variety"].dropna().unique().tolist()

def extract_variety(name):
    for v in variety_list:
        if isinstance(name, str) and v.lower() in name.lower():
            return v
    return "Unknown"

df["Variety"] = df["Name"].apply(extract_variety).fillna("Unknown")

# Drop missing
df = df.dropna(subset=["Rating", "Price", "Country", "Region", "Winery", "WineStyle", "Age"])

# Create SmoothedRating
global_mean = df["Rating"].mean()
df["SmoothedRating"] = (df["Rating"] * df["NumberOfRatings"] + global_mean * 5) / (df["NumberOfRatings"] + 5)

# Assign ReliabilityGroup
def reliability_group(n):
    if n < 35:
        return "Low"
    elif n < 838:
        return "Medium"
    else:
        return "High"

df["ReliabilityGroup"] = df["NumberOfRatings"].apply(reliability_group)

# Basic features
df["LogPrice"]            = np.log1p(df["Price"])
df["PricePerAge"]         = df["Price"] / (df["Age"] + 1)
df["IsOld"]               = df["Age"] > 10
df["WineryFreq"]          = df["Winery"].map(df["Winery"].value_counts() / len(df))
df["RegionFreq"]          = df["Region"].map(df["Region"].value_counts() / len(df))
df["CountryFreq"]         = df["Country"].map(df["Country"].value_counts() / len(df))
df["PriceAgeInteraction"] = df["PricePerAge"] * df["LogPrice"]
df["LogLogPrice"]         = np.log1p(df["LogPrice"])

# ————— Thêm VarietyFreq (tần suất xuất hiện của mỗi giống nho) —————
df["VarietyFreq"] = df["Variety"].map(df["Variety"].value_counts() / len(df))

# ===== 1.1. Country-level: tính trung bình SmoothedRating, xếp hạng, chuẩn hóa Score ∈ [0,1] =====
country_agg = (
    df.groupby("Country")["SmoothedRating"]
      .mean()
      .reset_index()
      .rename(columns={"SmoothedRating": "Country_MeanRating"})
)
country_agg["Country_Rank"] = country_agg["Country_MeanRating"]\
    .rank(method="first", ascending=False).astype(int)

max_rank_country = country_agg["Country_Rank"].max()
if max_rank_country > 1:
    country_agg["Country_Score"] = (max_rank_country - country_agg["Country_Rank"]) / (max_rank_country - 1)
else:
    # Nếu chỉ có duy nhất 1 country, gán score = 1.0 cho tất cả
    country_agg["Country_Score"] = 1.0

df = df.merge(
    country_agg[["Country", "Country_MeanRating", "Country_Rank", "Country_Score"]],
    how="left", on="Country"
)

# ===== 1.2. Region-level =====
region_agg = (
    df.groupby("Region")["SmoothedRating"]
      .mean()
      .reset_index()
      .rename(columns={"SmoothedRating": "Region_MeanRating"})
)
region_agg["Region_Rank"] = region_agg["Region_MeanRating"]\
    .rank(method="first", ascending=False).astype(int)

max_rank_region = region_agg["Region_Rank"].max()
if max_rank_region > 1:
    region_agg["Region_Score"] = (max_rank_region - region_agg["Region_Rank"]) / (max_rank_region - 1)
else:
    region_agg["Region_Score"] = 1.0

df = df.merge(
    region_agg[["Region", "Region_MeanRating", "Region_Rank", "Region_Score"]],
    how="left", on="Region"
)

# ===== 1.3. Winery-level =====
winery_agg = (
    df.groupby("Winery")["SmoothedRating"]
      .mean()
      .reset_index()
      .rename(columns={"SmoothedRating": "Winery_MeanRating"})
)
winery_agg["Winery_Rank"] = winery_agg["Winery_MeanRating"]\
    .rank(method="first", ascending=False).astype(int)

max_rank_winery = winery_agg["Winery_Rank"].max()
if max_rank_winery > 1:
    winery_agg["Winery_Score"] = (max_rank_winery - winery_agg["Winery_Rank"]) / (max_rank_winery - 1)
else:
    winery_agg["Winery_Score"] = 1.0

df = df.merge(
    winery_agg[["Winery", "Winery_MeanRating", "Winery_Rank", "Winery_Score"]],
    how="left", on="Winery"
)

# ===== 1.4. Variety-level =====
variety_agg = (
    df.groupby("Variety")["SmoothedRating"]
      .mean()
      .reset_index()
      .rename(columns={"SmoothedRating": "Variety_MeanRating"})
)
variety_agg["Variety_Rank"] = variety_agg["Variety_MeanRating"]\
    .rank(method="first", ascending=False).astype(int)

max_rank_variety = variety_agg["Variety_Rank"].max()
if max_rank_variety > 1:
    variety_agg["Variety_Score"] = (max_rank_variety - variety_agg["Variety_Rank"]) / (max_rank_variety - 1)
else:
    variety_agg["Variety_Score"] = 1.0

df = df.merge(
    variety_agg[["Variety", "Variety_MeanRating", "Variety_Rank", "Variety_Score"]],
    how="left", on="Variety"
)

# ===== 2. Kiểm tra và điền NaN nếu có =====
# Sau khi merge, đôi khi một số dòng (ví dụ trong holdout) chưa có match => Score = NaN
df["Country_Score"].fillna(df["Country_Score"].mean(), inplace=True)
df["Region_Score"].fillna(df["Region_Score"].mean(), inplace=True)
df["Winery_Score"].fillna(df["Winery_Score"].mean(), inplace=True)
df["Variety_Score"].fillna(df["Variety_Score"].mean(), inplace=True)

# (Nếu bạn muốn điền thêm các cột _MeanRating hay _Rank cũng có thể fillna tương tự)
df["Country_MeanRating"].fillna(df["Country_MeanRating"].mean(), inplace=True)
df["Region_MeanRating"].fillna(df["Region_MeanRating"].mean(), inplace=True)
df["Winery_MeanRating"].fillna(df["Winery_MeanRating"].mean(), inplace=True)
df["Variety_MeanRating"].fillna(df["Variety_MeanRating"].mean(), inplace=True)

df["Country_Rank"].fillna(df["Country_Rank"].median(), inplace=True)
df["Region_Rank"].fillna(df["Region_Rank"].median(), inplace=True)
df["Winery_Rank"].fillna(df["Winery_Rank"].median(), inplace=True)
df["Variety_Rank"].fillna(df["Variety_Rank"].median(), inplace=True)

# Các cột Frequency và other numeric nếu có NaN (ít khi xảy ra) cũng có thể fill 0:
df["WineryFreq"].fillna(0, inplace=True)
df["RegionFreq"].fillna(0, inplace=True)
df["CountryFreq"].fillna(0, inplace=True)
df["VarietyFreq"].fillna(0, inplace=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_12944\3128301413.py:161: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Country_Score"].fillna(df["Country_Score"].mean(), inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12944\3128301413.py:162: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

In [5]:
# ===== Define features and target =====
features = ["Country", "Region", "Winery", "Price", "WineStyle", "Variety",
            "Age", "PricePerAge", "IsOld", "LogPrice",
            "WineryFreq", "RegionFreq", "CountryFreq",
            "PriceAgeInteraction", "LogLogPrice", "Country_Score", "Region_Score", 
            "Winery_Score", "Variety_Score"]
target = "SmoothedRating"

# ===== Split into Mid, Low, High =====
mid_df = df[df["ReliabilityGroup"] == "Medium"].dropna(subset=features + [target]).copy()
low_df = df[df["ReliabilityGroup"] == "Low"].dropna(subset=features + [target]).copy()
high_df = df[df["ReliabilityGroup"] == "High"].dropna(subset=features + [target]).copy()

print("Mid – missing per feature:")
print(mid_df[features].isna().sum())

# ===== Split Mid into train and holdout =====
X_mid = mid_df[features]
y_mid = mid_df[target]

X_mid_train, X_mid_holdout, y_mid_train, y_mid_holdout = train_test_split(
    X_mid, y_mid, test_size=0.2, random_state=42
)

Mid – missing per feature:
Country                0
Region                 0
Winery                 0
Price                  0
WineStyle              0
Variety                0
Age                    0
PricePerAge            0
IsOld                  0
LogPrice               0
WineryFreq             0
RegionFreq             0
CountryFreq            0
PriceAgeInteraction    0
LogLogPrice            0
Country_Score          0
Region_Score           0
Winery_Score           0
Variety_Score          0
dtype: int64


In [6]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def safe_label_encode(df_train, df_apply, cat_cols):
    df_train_enc, encoders = encode_categorical(df_train, cat_cols)
    df_apply_enc = df_apply.copy()
    for col in cat_cols:
        known_labels = list(encoders[col].classes_)
        df_apply_enc[col] = df_apply_enc[col].astype(str).apply(
            lambda x: known_labels.index(x) if x in known_labels else -1
        )
    return df_train_enc, df_apply_enc

# ===== MID =====
cat_feats_mid = X_mid.select_dtypes(include="object").columns.tolist()
X_mid_train_enc, encoders_mid = encode_categorical(X_mid_train, cat_feats_mid)
X_mid_holdout_enc = X_mid_holdout.copy()
for col in cat_feats_mid:
    X_mid_train_enc, X_mid_holdout_enc = safe_label_encode(X_mid_train, X_mid_holdout, cat_feats_mid)


model_mid_only = XGBRegressor(
    n_estimators=500,     
    max_depth=3,              
    learning_rate=0.05,      
    subsample=0.7,         
    colsample_bytree=0.7,   
    min_child_weight=5,   
    reg_alpha=1.0,
    reg_lambda=1.0, 
    random_state=42,
    verbosity=0
)
model_mid_only.fit(X_mid_train_enc, y_mid_train)

y_pred_mid_holdout = model_mid_only.predict(X_mid_holdout_enc)
y_pred_mid_train = model_mid_only.predict(X_mid_train_enc)

mae_mid = mean_absolute_error(y_mid_holdout, y_pred_mid_holdout)
rmse_mid = mean_squared_error(y_mid_holdout, y_pred_mid_holdout)**0.5
r2_mid = r2_score(y_mid_holdout, y_pred_mid_holdout)
mask = y_mid_holdout != 0
mape_mid = (np.abs((y_mid_holdout[mask] - y_pred_mid_holdout[mask]) / y_mid_holdout[mask])).mean() * 100
print(f"Mid Holdout – MAE={mae_mid:.4f}, RMSE={rmse_mid:.4f}, R²={r2_mid:.4f}, MAPE={mape_mid:.2f}%")

mae_mid_train = mean_absolute_error(y_mid_train, y_pred_mid_train)
rmse_mid_train = mean_squared_error(y_mid_train, y_pred_mid_train)**0.5
r2_mid_train = r2_score(y_mid_train, y_pred_mid_train)
mask_mid_train = y_mid_train != 0
mape_mid_train = (np.abs((y_mid_train[mask_mid_train] - y_pred_mid_train[mask_mid_train]) / y_mid_train[mask_mid_train])).mean() * 100
print(f"Mid Train – MAE={mae_mid_train:.4f}, RMSE={rmse_mid_train:.4f}, R²={r2_mid_train:.4f}, MAPE={mape_mid_train:.2f}%")
print()

# ===== LOW =====
features_low = features.copy()
X_low = low_df[features_low]
y_low = low_df[target]
X_tr_low, X_va_low, y_tr_low, y_va_low = train_test_split(X_low, y_low, test_size=0.2, random_state=42)

cat_feats_low = X_low.select_dtypes(include="object").columns.tolist()
X_tr_low_enc, encoders_low = encode_categorical(X_tr_low, cat_feats_low)
X_tr_low_enc, X_va_low_enc = safe_label_encode(X_tr_low, X_va_low, cat_feats_low)

model_low_improved = XGBRegressor(
    n_estimators=500,     
    max_depth=3,              
    learning_rate=0.05,      
    subsample=0.7,         
    colsample_bytree=0.7,   
    min_child_weight=5,   
    reg_alpha=1.0,
    reg_lambda=1.0, 
    random_state=42,
    verbosity=0
)
model_low_improved.fit(X_tr_low_enc, y_tr_low)

y_pred_va_low = model_low_improved.predict(X_va_low_enc)
mae_va_low = mean_absolute_error(y_va_low, y_pred_va_low)
rmse_va_low = mean_squared_error(y_va_low, y_pred_va_low)**0.5
r2_va_low = r2_score(y_va_low, y_pred_va_low)
mask = y_va_low != 0
mape_va_low = (np.abs((y_va_low[mask] - y_pred_va_low[mask]) / y_va_low[mask])).mean() * 100
print(f"Low Valid – MAE={mae_va_low:.4f}, RMSE={rmse_va_low:.4f}, R²={r2_va_low:.4f}, MAPE={mape_va_low:.2f}%")

y_pred_low_train = model_low_improved.predict(X_tr_low_enc)
mae_low_train = mean_absolute_error(y_tr_low, y_pred_low_train)
rmse_low_train = mean_squared_error(y_tr_low, y_pred_low_train)**0.5
r2_low_train = r2_score(y_tr_low, y_pred_low_train)
mask_low_train = y_tr_low != 0
mape_low_train = (np.abs((y_tr_low[mask_low_train] - y_pred_low_train[mask_low_train]) / y_tr_low[mask_low_train])).mean() * 100
print(f"Low Train – MAE={mae_low_train:.4f}, RMSE={rmse_low_train:.4f}, R²={r2_low_train:.4f}, MAPE={mape_low_train:.2f}%")
print()

# ===== HIGH =====
features_high = features.copy()
X_high_train_val = high_df[features_high]
y_high_train_val = high_df[target]
X_tr_high, X_va_high, y_tr_high, y_va_high = train_test_split(X_high_train_val, y_high_train_val, test_size=0.2, random_state=42)

cat_feats_high = X_high_train_val.select_dtypes(include="object").columns.tolist()
X_tr_high_enc, encoders_high = encode_categorical(X_tr_high, cat_feats_high)
X_tr_high_enc, X_va_high_enc = safe_label_encode(X_tr_high, X_va_high, cat_feats_high)

model_high_improved = XGBRegressor(
    n_estimators=500,     
    max_depth=3,              
    learning_rate=0.05,      
    subsample=0.7,         
    colsample_bytree=0.7,   
    min_child_weight=5,   
    reg_alpha=1.0,
    reg_lambda=1.0, 
    random_state=42,
    verbosity=0
)
model_high_improved.fit(X_tr_high_enc, y_tr_high)

y_pred_va_high = model_high_improved.predict(X_va_high_enc)
mae_va_high = mean_absolute_error(y_va_high, y_pred_va_high)
rmse_va_high = mean_squared_error(y_va_high, y_pred_va_high)**0.5
r2_va_high = r2_score(y_va_high, y_pred_va_high)
mask = y_va_high != 0
mape_va_high = (np.abs((y_va_high[mask] - y_pred_va_high[mask]) / y_va_high[mask])).mean() * 100
print(f"High Valid – MAE={mae_va_high:.4f}, RMSE={rmse_va_high:.4f}, R²={r2_va_high:.4f}, MAPE={mape_va_high:.2f}%")

y_pred_high_train = model_high_improved.predict(X_tr_high_enc)
mae_high_train = mean_absolute_error(y_tr_high, y_pred_high_train)
rmse_high_train = mean_squared_error(y_tr_high, y_pred_high_train)**0.5
r2_high_train = r2_score(y_tr_high, y_pred_high_train)
mask_high_train = y_tr_high != 0
mape_high_train = (np.abs((y_tr_high[mask_high_train] - y_pred_high_train[mask_high_train]) / y_tr_high[mask_high_train])).mean() * 100
print(f"High Train – MAE={mae_high_train:.4f}, RMSE={rmse_high_train:.4f}, R²={r2_high_train:.4f}, MAPE={mape_high_train:.2f}%")


Mid Holdout – MAE=0.1085, RMSE=0.1422, R²=0.7392, MAPE=2.83%
Mid Train – MAE=0.1012, RMSE=0.1322, R²=0.7764, MAPE=2.65%

Low Valid – MAE=0.1207, RMSE=0.1612, R²=0.6322, MAPE=3.24%
Low Train – MAE=0.0915, RMSE=0.1208, R²=0.7931, MAPE=2.43%

High Valid – MAE=0.0996, RMSE=0.1375, R²=0.7913, MAPE=2.60%
High Train – MAE=0.0730, RMSE=0.0932, R²=0.8930, MAPE=1.87%


In [7]:
# ===== ĐỊNH NGHĨA HÀM SMART_PREDICT DÙNG XGBOOST =====

def encode_row(row, encoder_dict, feature_list):
    """Encode một dòng dữ liệu với encoder đã học"""
    encoded = {}
    for col in feature_list:
        val = str(row[col])
        if col in encoder_dict:
            classes = encoder_dict[col].classes_
            encoded[col] = classes.tolist().index(val) if val in classes else -1
        else:
            encoded[col] = row[col]
    return pd.DataFrame([encoded])

def smart_predict(row):
    """
    row: pandas Series chứa đủ các cột trong features/ features_low / features_high + cột 'ReliabilityGroup'
    Trả về:
      - Nếu row['ReliabilityGroup'] == 'Low': blend(mid, low)
      - Nếu row['ReliabilityGroup'] == 'High': blend(mid, high)
      - Nếu row['ReliabilityGroup'] == 'Medium': mid-only
    """
    # Luôn tính pred_mid
    df_tmp_mid = encode_row(row, encoders_mid, features)
    pred_mid = model_mid_only.predict(df_tmp_mid)[0]

    if row["ReliabilityGroup"] == "Low":
        df_tmp_low = encode_row(row, encoders_low, features_low)
        pred_low = model_low_improved.predict(df_tmp_low)[0]
        return 0.3 * pred_mid + 0.7 * pred_low

    if row["ReliabilityGroup"] == "High":
        df_tmp_high = encode_row(row, encoders_high, features_high)
        pred_high = model_high_improved.predict(df_tmp_high)[0]
        return 0.3 * pred_mid + 0.7 * pred_high

    return pred_mid


In [8]:
# ===== TẠO test_df KẾT HỢP MID_HOLDOUT, LOW, HIGH =====

mid_holdout_df = X_mid_holdout.copy()
mid_holdout_df["ReliabilityGroup"] = "Medium"
mid_holdout_df["TrueRating"] = y_mid_holdout.values

low_holdout_df = low_df[features_low].copy()
low_holdout_df["ReliabilityGroup"] = "Low"
low_holdout_df["TrueRating"] = low_df[target].values

high_holdout_df = high_df[features_high].copy()
high_holdout_df["ReliabilityGroup"] = "High"
high_holdout_df["TrueRating"] = high_df[target].values

test_df = pd.concat([mid_holdout_df, low_holdout_df, high_holdout_df], ignore_index=True)
test_df["PredictedRating"] = test_df.apply(smart_predict, axis=1)

# ===== TÍNH METRICS TOÀN BỘ =====

mae_all  = mean_absolute_error(test_df["TrueRating"], test_df["PredictedRating"])
rmse_all = mean_squared_error(test_df["TrueRating"], test_df["PredictedRating"]) ** 0.5
r2_all   = r2_score(test_df["TrueRating"], test_df["PredictedRating"])
mask_all = test_df["TrueRating"] != 0
mape_all = (np.abs((test_df["TrueRating"][mask_all] - test_df["PredictedRating"][mask_all]) / test_df["TrueRating"][mask_all])).mean() * 100

print(f"All Groups      – MAE={mae_all:.4f}, RMSE={rmse_all:.4f}, R²={r2_all:.4f}, MAPE={mape_all:.2f}%")

# ===== METRICS RIÊNG THEO NHÓM =====

for grp in ["Low", "Medium", "High"]:
    sub = test_df[test_df["ReliabilityGroup"] == grp]
    if len(sub) == 0:
        continue
    mae_g  = mean_absolute_error(sub["TrueRating"], sub["PredictedRating"])
    rmse_g = mean_squared_error(sub["TrueRating"], sub["PredictedRating"]) ** 0.5
    r2_g   = r2_score(sub["TrueRating"], sub["PredictedRating"])
    mask_g = sub["TrueRating"] != 0
    mape_g = (np.abs((sub["TrueRating"][mask_g] - sub["PredictedRating"][mask_g]) / sub["TrueRating"][mask_g])).mean() * 100
    print(f"{grp:6s} Group     – MAE={mae_g:.4f}, RMSE={rmse_g:.4f}, R²={r2_g:.4f}, MAPE={mape_g:.2f}%")


All Groups      – MAE=0.0993, RMSE=0.1317, R²=0.7813, MAPE=2.60%
Low    Group     – MAE=0.1016, RMSE=0.1358, R²=0.7389, MAPE=2.71%
Medium Group     – MAE=0.1085, RMSE=0.1422, R²=0.7392, MAPE=2.83%
High   Group     – MAE=0.0821, RMSE=0.1077, R²=0.8603, MAPE=2.11%


In [17]:
def print_error_rate_by_group(test_df, group_name, threshold=0.2):
    df_group = test_df[test_df["ReliabilityGroup"] == group_name].copy()
    if df_group.shape[0] == 0:
        print(f"Không tìm thấy nhóm '{group_name}' trong test_df.")
        return

    df_group["abs_residual"] = np.abs(df_group["TrueRating"] - df_group["PredictedRating"])
    num_errors = (df_group["abs_residual"] > threshold).sum()
    total = df_group.shape[0]
    percent_error = num_errors / total * 100

    print(f"Nhóm {group_name}: có {num_errors} mẫu (trên {total}) có |residual| > {threshold}, "
          f"tương đương {percent_error:.2f}%")

print_error_rate_by_group(test_df, "High", threshold=0.2)
print_error_rate_by_group(test_df, "Medium", threshold=0.2)
print_error_rate_by_group(test_df, "Low", threshold=0.2)

Nhóm High: có 86 mẫu (trên 1370) có |residual| > 0.2, tương đương 6.28%
Nhóm Medium: có 318 mẫu (trên 2219) có |residual| > 0.2, tương đương 14.33%
Nhóm Low: có 180 mẫu (trên 1369) có |residual| > 0.2, tương đương 13.15%


In [20]:
import os, joblib
os.makedirs("models", exist_ok=True)

# Lưu mô hình
joblib.dump(model_mid_only,      "models/XGBoost_Mid.pkl")
joblib.dump(model_low_improved,  "models/XGBoost_Low.pkl")
joblib.dump(model_high_improved, "models/XGBoost_High.pkl")

# Lưu encoder tương ứng
joblib.dump(encoders_mid,        "models/XGBoost_Encoders_Mid.pkl")
joblib.dump(encoders_low,        "models/XGBoost_Encoders_Low.pkl")
joblib.dump(encoders_high,       "models/XGBoost_Encoders_High.pkl")


['models/XGBoost_Encoders_High.pkl']